In [1]:
import openmc
import numpy as np

For our infinite medium calculations, we'll use three different materials: water, UO$_2$, and a mixture of water and UO$_2$.

First, let's define the UO$_2$, which we'll take to be 4 w/o enriched (i.e., of the uranium mass, 4% is U-235).  OpenMC supports a variety of ways to define materials; here, we'll use mass fractions and set the mass density.

In [2]:
MUO2 = 238*0.96 + 235*0.04 + 32.0
MU = 238*0.96 + 235*0.04
uo2 = openmc.Material(name="uo2")
uo2.add_nuclide('U235', 0.04*MU/MUO2, 'wo')
uo2.add_nuclide('U238', 0.96*MU/MUO2, 'wo')
uo2.add_nuclide('O16',  (MUO2-MU)/MUO2, 'wo')
uo2.set_density('g/cm3', 10.0)

To check that the resulting material is correct, note that the number density (in 1/b-cm) of U235 should be

In [3]:
0.04*MU/MUO2*10.0/235*6.022e23*1e-24

0.0009034832565458566

We can inspect the number densities computed by OpenMC via

In [4]:
uo2.get_nuclide_atom_densities()

OrderedDict([('U235', 0.000903335515705231),
             ('U238', 0.021406208054767537),
             ('O16', 0.04464247386494846)])

from which we can determine the numbers are consistent (the difference being from OpenMC's use of the actual atomic masses of the nuclides, e.g., 238.0507 and change instead of 238).

Now, for the water and fuel/water mixture:

In [5]:
water = openmc.Material(name="h2o")
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3', 1.0)

In [6]:
uo2_water = openmc.Material.mix_materials([uo2, water], [0.5, 0.5], 'vo')

With these three materials defined, we can produce the `materials.xml` file required by OpenMC for simulations that need these materials.

In [7]:
materials = openmc.Materials([uo2, water, uo2_water])
materials.export_to_xml()